# Scraping

### Quelques autres méthodes du type `str`

### Cleaning

In [1]:
# Ma string s contient:
# - des espaces
# - des tabulations (\t)
# - des retours à la ligne (\n)
s = "\t hello\nworld  "
print(s)

	 hello
world  


In [2]:
# Remplacer le retour à la ligne par un espace:
print(s.replace('\n', ' '))

	 hello world  


In [3]:
# Enlever les caractères blancs au début et à la fin de la string:
print(s.strip())

hello
world


In [4]:
cleaned = s.replace('\n', ' ').strip()
print(cleaned)

hello world


### Trouver une str dans une autre

In [5]:
cleaned.startswith('he')

True

In [6]:
cleaned.endswith('orld')

True

In [7]:
"wo" in cleaned

True

In [8]:
cleaned.index("wor")  # renvoie la position de "wor" dans "hello world"

6

### str.join

Permet de joindre une liste de strings.

In [9]:
" ".join(['how', 'are', 'you'])

'how are you'

In [10]:
"---".join(['how', 'are', 'you'])

'how---are---you'

Dans jupyterlab, pensez à utiliser `<Tab>` pour l'autocomplétion et pour découvrir quelles méthodes un type possède. `<Shift>` + `<Tab>` pour afficher la documentation d'une fonction.

Je vous invite aussi à survoler les fonctions présentes de base en python:
- https://docs.python.org/3/library/functions.html

Ainsi que les modules présents de base (stdlib):
- https://docs.python.org/3/library/

Et pour tout le reste il y a pypi:
- http://pypi.org

In [11]:
# "pip install requests" pour pouvoir l'utiliser
import requests

response = requests.get('https://fr.wikipedia.org/plop')
print(response)

<Response [404]>


In [12]:
print(response.status_code)

404


In [13]:
page = "https://www.beerwulf.com/fr-fr/p/bieres/brasserie-de-sutter-brin-de-folie.33"
content = requests.get(page).text
print(content[:100])


<!doctype html>
<html class="no-js" lang="fr-FR"
      data-original-lang="fr-FR"
      data-re


Pour voir le code source HTML d'une page dans le navigateur:
- Ctrl-U
- ou F12 > Elements

In [14]:
before_price = '<span class="price">'
idx = content.index(before_price)
price = content[idx+len(before_price):idx+100]
price = price.split('<')[0]

In [15]:
price

'€ 2,29'

In [16]:
print(float(price[2:].replace(',', '.')))

2.29


### Moralité
C'est un peu le bordel d'extraire des infos du document html en manipulant le document comme une bête str.

Here comes beautifulsoup:

In [17]:
from bs4 import BeautifulSoup

Je vous invite à tester le HTML suivant sur https://html.house.

([Cours HTML/CSS sur openclassrooms](https://openclassrooms.com/fr/courses/1603881-apprenez-a-creer-votre-site-web-avec-html5-et-css3))

In [18]:
html = """
<html>
    <head>
        <style>
        h1 { font-size: 50px; }
        body { font-family: Verdana; }
        li { color: red; }
        ul ul li { color: green; }
        .highlighted { font-weight: bold; }
        .italic { font-style: italic; }
        .highlighted.italic { }
        </style>
    </head>
    <body>
        <h1>Mon titre</h1>
        <p class="highlighted">
            Some text with a<br>
            <a href="https://google.com">link to google</a>
            <img src="https://picsum.photos/200/300">
        </p>
        <p>Some list:</p>
        <ul>
            <li>some item</li>
            <li class="highlighted italic">some item</li>
            <li class="italic">some item</li>
            <ul>
                <li>some other item 1</li>
                <li>some other item 2</li>
            </ul>
            <li>some item</li>
        </ul>
    </body>
</html>
"""

In [19]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html)

In [20]:
titre = soup.find('h1')

In [21]:
titre

<h1>Mon titre</h1>

In [22]:
type(titre)

bs4.element.Tag

In [23]:
titre.text

'Mon titre'

In [24]:
titre.name

'h1'

In [25]:
link = soup.find('a')
link.attrs

{'href': 'https://google.com'}

In [26]:
paragraph = soup.find('p')

In [27]:
paragraph

<p class="highlighted">
            Some text with a<br/>
<a href="https://google.com">link to google</a>
<img src="https://picsum.photos/200/300"/>
</p>

In [28]:
paragraph.find('img')

<img src="https://picsum.photos/200/300"/>

In [29]:
soup.find_all('li', class_="italic")

[<li class="highlighted italic">some item</li>,
 <li class="italic">some item</li>]

In [30]:
# La même chose qu'au dessus, mais à l'aide d'un sélecteur css:
soup.select('li.italic')

[<li class="highlighted italic">some item</li>,
 <li class="italic">some item</li>]

In [31]:
# Récupérer les li de 2e niveau (qui sont dans un ul lui-même dans un ul)
soup.find('ul').find('ul').find_all('li')

[<li>some other item 1</li>, <li>some other item 2</li>]

In [32]:
# Même chose avec un sélecteur css:
soup.select('ul ul li')

[<li>some other item 1</li>, <li>some other item 2</li>]

In [33]:
li = soup.select('ul ul li')[0]
li

<li>some other item 1</li>

In [34]:
li.find_next_sibling()

<li>some other item 2</li>

In [35]:
li.parent

<ul>
<li>some other item 1</li>
<li>some other item 2</li>
</ul>

In [36]:
li.parent.contents

['\n', <li>some other item 1</li>, '\n', <li>some other item 2</li>, '\n']

In [37]:
li.parent.find_all()  # même chose que .contents mais renvoie uniquement les Tag

[<li>some other item 1</li>, <li>some other item 2</li>]

### Cas concret: beerwulf.com

In [38]:
from bs4 import BeautifulSoup

def get_soup_from_url(url):
    page = requests.get(url)
    return BeautifulSoup(page.text)
    
def extract_beer_infos(url):
    soup = get_soup_from_url(url)
    
    # Extract price:
    price = soup.select('span.price')[0].text
    price = float(price[2:].replace(',', '.'))  # "€ 2,29" => 2.29
    
    # Extract volume:
    volume = soup.find('dt', text='Contenu').find_next_sibling()
    volume = int(volume.text[:-2])  # "33cl" => 33
    
    # Extract evaluation:
    note = soup.find('div', class_='stars')
    note = int(note.attrs['data-percent'])
    
    # Extract EBC:
    ebc = soup.find('div', class_='ebc')
    children = ebc.find_all('div')
    active_tag = ebc.find('div', class_='active')
    position = children.index(active_tag)
    ebc_pct = position / len(children) * 100
    
    infos = {
        'price': price,
        'volume': volume,
        'note': note,
        'ebc': ebc_pct,
    }
    return infos

In [39]:
extract_beer_infos("https://www.beerwulf.com/fr-fr/p/bieres/brugse-zot-blond2")

{'price': 2.29, 'volume': 33, 'note': 70, 'ebc': 7.6923076923076925}

### Inconvénients du scraping:

1) le contenu généré dynamiquement en javascript n'est pas présent initialement sur la page, ce qui fait qu'on peut avoir un contenu différent entre ce qu'on trouve dans l'inspecteur (sur chrome: F12 > Elements)

In [40]:
soup = get_soup_from_url("https://www.beerwulf.com/fr-fr/c/bieres/style/Blonde")

In [41]:
# Ce find ne renvoie rien car le products-container est ajouté via Javascript
soup.find('div', class_="products-container")

Un exemple minimal de page où le code source de la page ne contient pas les données initialement: https://kim.fspot.org/cours/page4.html

In [42]:
infos = [
    extract_beer_infos('https://www.beerwulf.com/fr-fr/p/bieres/brouwerij-t-verzet-super-noah.33'),
    extract_beer_infos('https://www.beerwulf.com/fr-fr/p/bieres/bruxellensis2')
]

In [43]:
infos

[{'price': 1.99, 'volume': 33, 'note': 70, 'ebc': 15.384615384615385},
 {'price': 3.49, 'volume': 33, 'note': 70, 'ebc': 15.384615384615385}]

In [44]:
import pandas as pd
df = pd.DataFrame(infos)

In [45]:
df

,ebc,note,price,volume
0,15.384615,70,1.99,33
1,15.384615,70,3.49,33


In [46]:
df[df.price < 2]

,ebc,note,price,volume
0,15.384615,70,1.99,33


2) inconvénient du scraping: on est tributaire de l'architecture du HTML. Si les développeurs du site web changent le design, il y a de fortes chances que le programme beautifulsoup doive être réécrit.

Solution: **préférer une API**